In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import statsmodels.api as sm

In [3]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, Lars
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
automobile_df = pd.read_csv('automobile_df_processed.csv')
automobile_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,age
0,27.2,4,119.0,97.0,2300,14.7,44
1,13.0,8,350.0,145.0,3988,13.0,49
2,32.4,4,107.0,72.0,2290,17.0,42
3,15.0,8,350.0,145.0,4082,13.0,49
4,13.0,8,360.0,175.0,3821,11.0,49


In [7]:
result_dict = {}

In [ ]:
def build_model(regression_fn, col_y_name, col_x_names, dataset, test_frac=.2, show_plot_y = False, show_scatter = False):
    X = dataset[col_x_names]
    y = dataset[col_y_name]
    